In [1]:
from scripts.flink_sql_gateway_wrapper import connect, sql, close
from time import sleep

In [2]:
connect("localhost", 8081)

✓ Connected to SQL Gateway at localhost:8081


In [3]:
sql("SHOW CATALOGS")

✓ Loaded 1 rows into table: result
┌─────────────────┐
│  catalog name   │
│     varchar     │
├─────────────────┤
│ default_catalog │
└─────────────────┘



In [4]:
sql("""CREATE CATALOG IF NOT EXISTS paimon_catalog WITH (
  'type' = 'paimon',
  'warehouse' = 's3://paimon-data/paimon-warehouse',
  's3.endpoint' = 'http://seaweedfs-s3:8333',
  's3.access-key' = 'paimonAdmin123',
  's3.secret-key' = 'paimonSecretKey456789abcdef',
  's3.path-style-access' = 'true'
);""")

✓ Loaded 1 rows into table: result
┌─────────┐
│ result  │
│ varchar │
├─────────┤
│ OK      │
└─────────┘



In [5]:
sql("USE CATALOG paimon_catalog")
sql("SHOW DATABASES")

✓ Loaded 1 rows into table: result
┌─────────┐
│ result  │
│ varchar │
├─────────┤
│ OK      │
└─────────┘

✓ Loaded 1 rows into table: result
┌───────────────┐
│ database name │
│    varchar    │
├───────────────┤
│ default       │
└───────────────┘



In [6]:
sql("""CREATE DATABASE IF NOT EXISTS testdb;""")
sql("SHOW DATABASES")
sql("""USE testdb;""")
sql("SHOW TABLES")

✓ Loaded 1 rows into table: result
┌─────────┐
│ result  │
│ varchar │
├─────────┤
│ OK      │
└─────────┘

✓ Loaded 2 rows into table: result
┌───────────────┐
│ database name │
│    varchar    │
├───────────────┤
│ default       │
│ testdb        │
└───────────────┘

✓ Loaded 1 rows into table: result
┌─────────┐
│ result  │
│ varchar │
├─────────┤
│ OK      │
└─────────┘

✓ Query returned 0 rows


In [7]:
sql("DROP TABLE IF EXISTS users;")
sql("DROP TABLE IF EXISTS orders;")

✓ Loaded 1 rows into table: result
┌─────────┐
│ result  │
│ varchar │
├─────────┤
│ OK      │
└─────────┘

✓ Loaded 1 rows into table: result
┌─────────┐
│ result  │
│ varchar │
├─────────┤
│ OK      │
└─────────┘



In [7]:
sql("""CREATE TEMPORARY TABLE IF NOT EXISTS mysql_users (
  id INT,
  name STRING,
  email STRING,
  age INT,
  created_at TIMESTAMP(3),
  updated_at TIMESTAMP(3),
  PRIMARY KEY (id) NOT ENFORCED
) WITH (
  'connector' = 'mysql-cdc',
  'hostname' = 'mysql',
  'port' = '3306',
  'username' = 'flink',
  'password' = 'flink123',
  'database-name' = 'testdb',
  'table-name' = 'users',
  'scan.startup.mode' = 'initial'
);""")
sql("SELECT * FROM mysql_users;")

✓ Loaded 1 rows into table: result
┌─────────┐
│ result  │
│ varchar │
├─────────┤
│ OK      │
└─────────┘

⏳ Waiting for CDC snapshot to complete...
✗ No data received within timeout period
   This usually means:
   - CDC source is still initializing (increase timeout)
   - Source table is empty
   - Connection issues with source database
✓ Query returned 0 rows
✓ Stopped job aebaedf8cf1893840162fe1c54af8292


In [9]:
sql("SELECT * FROM mysql_users;")

✓ Loaded 5 rows into table: result
┌───────┬───────────────┬─────────────────────┬───────┬─────────────────────┬─────────────────────┐
│  id   │     name      │        email        │  age  │     created_at      │     updated_at      │
│ int32 │    varchar    │       varchar       │ int32 │       varchar       │       varchar       │
├───────┼───────────────┼─────────────────────┼───────┼─────────────────────┼─────────────────────┤
│     2 │ Bob Smith     │ bob@example.com     │    35 │ 2025-12-11T17:06:52 │ 2025-12-11T17:06:52 │
│     3 │ Charlie Brown │ charlie@example.com │    42 │ 2025-12-11T17:06:52 │ 2025-12-11T17:06:52 │
│     4 │ Diana Prince  │ diana@example.com   │    30 │ 2025-12-11T17:06:52 │ 2025-12-11T17:06:52 │
│     5 │ Eve Wilson    │ eve@example.com     │    25 │ 2025-12-11T17:06:52 │ 2025-12-11T17:06:52 │
│     1 │ Alice Johnson │ alice@example.com   │    28 │ 2025-12-11T17:06:52 │ 2025-12-11T17:06:52 │
└───────┴───────────────┴─────────────────────┴───────┴──────────

In [8]:
sql("""CREATE TEMPORARY TABLE IF NOT EXISTS mysql_orders (
  id INT,
  user_id INT,
  product_name STRING,
  quantity INT,
  price DECIMAL(10, 2),
  order_date TIMESTAMP(3),
  PRIMARY KEY (id) NOT ENFORCED
) WITH (
  'connector' = 'mysql-cdc',
  'hostname' = 'mysql',
  'port' = '3306',
  'username' = 'flink',
  'password' = 'flink123',
  'database-name' = 'testdb',
  'table-name' = 'orders',
  'scan.startup.mode' = 'initial'
);""")
sql("SELECT * FROM mysql_orders;")

✓ Loaded 1 rows into table: result
┌─────────┐
│ result  │
│ varchar │
├─────────┤
│ OK      │
└─────────┘

⏳ Waiting for data to become available...
✗ No data available after 30 seconds
✓ Stopped job c4e8dc635476756b18856723556ff001


In [13]:
sql("""CREATE TABLE IF NOT EXISTS users (
  id INT,
  name STRING,
  email STRING,
  age INT,
  created_at TIMESTAMP(4),
  updated_at TIMESTAMP(4),
  PRIMARY KEY (id) NOT ENFORCED
) WITH (
  'metadata.iceberg.storage' = 'table-location',
  'data-file.path-directory' = 'data'
);""")
sql("""INSERT INTO users SELECT * FROM mysql_users;""")

✓ Loaded 1 rows into table: result
┌─────────┐
│ result  │
│ varchar │
├─────────┤
│ OK      │
└─────────┘

✓ Loaded 1 rows into table: result
┌──────────────────────────────────┐
│              job id              │
│             varchar              │
├──────────────────────────────────┤
│ 18af80910e031ac9a16cda68b9eb672d │
└──────────────────────────────────┘



In [ ]:
sleep(30)
sql("SELECT * FROM users")

✓ Loaded 5 rows into table: result
┌───────┬───────────────┬─────────────────────┬───────┬─────────────────────┬─────────────────────┐
│  id   │     name      │        email        │  age  │     created_at      │     updated_at      │
│ int32 │    varchar    │       varchar       │ int32 │       varchar       │       varchar       │
├───────┼───────────────┼─────────────────────┼───────┼─────────────────────┼─────────────────────┤
│     1 │ Alice Johnson │ alice@example.com   │    28 │ 2025-12-11T17:06:52 │ 2025-12-11T17:06:52 │
│     2 │ Bob Smith     │ bob@example.com     │    35 │ 2025-12-11T17:06:52 │ 2025-12-11T17:06:52 │
│     3 │ Charlie Brown │ charlie@example.com │    42 │ 2025-12-11T17:06:52 │ 2025-12-11T17:06:52 │
│     4 │ Diana Prince  │ diana@example.com   │    30 │ 2025-12-11T17:06:52 │ 2025-12-11T17:06:52 │
│     5 │ Eve Wilson    │ eve@example.com     │    25 │ 2025-12-11T17:06:52 │ 2025-12-11T17:06:52 │
└───────┴───────────────┴─────────────────────┴───────┴──────────

In [14]:
sql("""CREATE TABLE IF NOT EXISTS orders (
  id INT,
  user_id INT,
  product_name STRING,
  quantity INT,
  price DECIMAL(10, 2),
  order_date TIMESTAMP(4),
  PRIMARY KEY (id) NOT ENFORCED
) WITH (
  'metadata.iceberg.storage' = 'table-location',
  'data-file.path-directory' = 'data'
);""")
sql("""INSERT INTO orders SELECT * FROM mysql_orders;""")

✓ Loaded 1 rows into table: result
┌─────────┐
│ result  │
│ varchar │
├─────────┤
│ OK      │
└─────────┘

✓ Loaded 1 rows into table: result
┌──────────────────────────────────┐
│              job id              │
│             varchar              │
├──────────────────────────────────┤
│ d157a5e7de130a0050cb3bfe8107c545 │
└──────────────────────────────────┘



In [ ]:
sleep(30)
sql("SELECT * FROM orders")

✓ Loaded 6 rows into table: result
┌───────┬─────────┬──────────────┬──────────┬────────┬─────────────────────┐
│  id   │ user_id │ product_name │ quantity │ price  │     order_date      │
│ int32 │  int32  │   varchar    │  int32   │ double │       varchar       │
├───────┼─────────┼──────────────┼──────────┼────────┼─────────────────────┤
│     1 │       1 │ Laptop       │        1 │ 999.99 │ 2025-12-11T17:06:52 │
│     2 │       2 │ Mouse        │        2 │   25.5 │ 2025-12-11T17:06:52 │
│     3 │       1 │ Keyboard     │        1 │   75.0 │ 2025-12-11T17:06:52 │
│     4 │       3 │ Monitor      │        1 │ 299.99 │ 2025-12-11T17:06:52 │
│     5 │       4 │ Headphones   │        1 │  150.0 │ 2025-12-11T17:06:52 │
│     6 │       5 │ USB Cable    │        3 │   10.0 │ 2025-12-11T17:06:52 │
└───────┴─────────┴──────────────┴──────────┴────────┴─────────────────────┘

✓ Stopped job c2daa484872f1e9823a84ad2af5a456e


In [19]:
close()